In [6]:
import fsspec
import s3fs
import re
import xarray as xr

def decade_month_calc(ds: xr.Dataset, time_dim: str = "time") -> xr.Dataset:
    """Calculates the climatological mean by decade and month.

    This function computes the decade-by-decade average for each month in the provided dataset.
    The process involves averaging values across each decade for each month separately.
    For instance, for the 2050s, the function calculates the average values for January, February,
    March, and so on, resulting in 12 averaged values corresponding to each month of the 2050s.
    This approach preserves seasonal variability while smoothing out interannual variability
    within each decade.

    The function performs the following steps:
    1. Assigns new coordinates to the dataset:
       - `decade`: Represents the decade (e.g., 2050 for the 2050s).
       - `month`: Represents the month (1 for January, 2 for February, etc.).
    2. Creates a combined `decade_month` coordinate, formatted as "YYYY-MM",
       where "YYYY" is the starting year of the decade, and "MM" is the month.
    3. Groups the dataset by the `decade_mon
    """
    ds = ds.assign_coords(
        decade=(ds["time.year"] // 10) * 10, month=ds["time"].dt.month
    )

    ds = ds.assign_coords(
        decade_month=(
            time_dim,
            [
                f"{decade}-{month:02d}"
                for decade, month in zip(ds["decade"].values, ds["month"].values)
            ],
        )
    )

    ds = ds.groupby("decade_month").mean()

    return ds

def extract_model_from_path(path):
    match = re.search(r"/([^/]+)/ssp\d+/", path)
    return match.group(1) if match else "unknown_model"

s3_bucket = "uw-crl"
s3_prefix = "climate-risk-map/backend/climate/scenariomip/NEX-GDDP-CMIP6"
ssp = "126"  # or ssp245, ssp370, etc.

# Initialize an anonymous S3 filesystem (assuming public data)
fs = s3fs.S3FileSystem()

# Build a wildcard pattern to find all relevant Zarr stores


pattern_1 = f"{s3_bucket}/{s3_prefix}/ACCESS-CM2/ssp{ssp}/*/*.zarr"
pattern_2 = f"{s3_bucket}/{s3_prefix}/ACCESS-ESM1-5/ssp{ssp}/*/*.zarr"

# Use fs.glob to list all Zarr stores matching the pattern
zarr_paths_1 = fs.glob(pattern_1)
zarr_paths_2 = fs.glob(pattern_2)

# if not zarr_paths:
#     print("No Zarr stores found for the given pattern.")
# else:
#     print(f"Found {len(zarr_paths)} Zarr stores")

# Use the full S3 URIs for Xarray
zarr_uris_1 = [f"s3://{path}" for path in zarr_paths_1]
zarr_uris_2 = [f"s3://{path}" for path in zarr_paths_2]

# Open multiple Zarr datasets
ds_1 = xr.open_mfdataset(zarr_uris_1, engine="zarr", combine="by_coords", data_vars=["fwi"], parallel=True, preprocess=decade_month_calc)
ds_2 = xr.open_mfdataset(zarr_uris_2, engine="zarr", combine="by_coords", data_vars=["fwi"], parallel=True, preprocess=decade_month_calc)

ds_1 = ds_1.assign_coords(model="ACCESS-CM2")
ds_1 = ds_1.expand_dims("model")

ds_2 = ds_2.assign_coords(model="ACCESS-ESM1-5")
ds_2 = ds_2.expand_dims("model")


In [14]:
ds_combined = xr.combine_nested([ds_1, ds_2], concat_dim=["model"])

In [29]:
ds_combined.fwi.sel(decade_month="2050-07").sel(lat=47, lon=290, method="nearest").compute()

<xarray.DataArray 'fwi' (model: 2)> Size: 8B
array([0.47432232, 0.41958573], dtype=float32)
Coordinates:
    lat           float64 8B 47.12
    lon           float64 8B 290.1
    decade_month  <U7 28B '2050-07'
  * model         (model) <U13 104B 'ACCESS-CM2' 'ACCESS-ESM1-5'

In [37]:
ds_combined.fwi.sel(decade_month="2050-07").sel(lat=47, method="nearest").max(dim="model").compute()

<xarray.DataArray 'fwi' (lon: 280)> Size: 1kB
array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan, 14.520293  , 16.857885  ,
       19.201044  , 21.70739   , 23.760263  , 25.710335  , 26.195126  ,
       26.849813  , 27.672918  , 26.658503  , 28.109678  , 29.43224   ,
       32.372063  , 35.446266  , 37.183132  , 37.19852   , 39.625767  ,
       41.531723  , 41.779377  , 41.92391   , 41.9216    , 41.661892  ,
       41.02065   , 40.724472  , 40.37713   , 39.31167   , 37.99311   ,
       36.421516  , 35.40726   , 34.656002  , 34.32684   , 34.041885  ,
       33.407097  , 32.89496   , 33.735058  , 34.26222   , 34.754177  ,
       37.416218  , 38.05265   , 39.0032    , 39.15736   , 35.76983   ,
       36.18259   , 35.62653   , 33.87061   , 32.219406  , 31.503597  ,
       32.453323  , 33.52883   , 32.41726   , 31.127     , 31.103218  ,
       29.21151   , 28.998695  , 28.229782  , 28.811922  , 29.137486  ,
       28.939245  , 28.84068   , 29.03707   , 30.41063   , 33.578976  ,
       34.87593   , 35.561913  , 36.18861   , 36.86698   , 36.77166   ,
       36.879833  , 36.6805    , 36.395042  , 35.741684  , 35.047844  ,
       34.052017  , 32.211086  , 31.849758  , 32.061214  , 32.92038   ,
...
        5.2994723 ,  5.3720875 ,  5.1500177 ,  4.964711  ,  4.7420135 ,
        4.62676   ,  4.4606733 ,  4.227566  ,  4.026168  ,  3.8673859 ,
        3.445203  ,  3.1690047 ,  2.9903839 ,  2.8875206 ,  2.8449395 ,
        2.9317484 ,  2.7216885 ,  2.4569688 ,  2.1214857 ,  2.032822  ,
        2.0009754 ,  1.9847856 ,  1.9776447 ,  1.9682379 ,  1.9627557 ,
        1.9423953 ,  1.937633  ,  1.9232918 ,  1.717607  ,  1.4928946 ,
        1.3260003 ,  1.1839869 ,  1.0972791 ,  0.9803443 ,  0.93117857,
        0.85771024,  0.72787875,  0.59586245,  0.5203576 ,  0.47200096,
        0.40235358,  0.35185993,  0.30707344,  0.2930441 ,  0.26665473,
        0.25155577,  0.23570678,  0.20148021,  0.19279993,  0.19831355,
        0.21863858,  0.28049132,  0.3378856 ,  0.45691448,  0.44473726,
        0.47432232,  0.45493814,  0.52169687,  0.56391317,  0.607698  ,
        0.71749127,  0.8546103 ,  0.83293605,  0.8483976 ,  0.87452227,
        0.90614504,  0.91941977,  0.948963  ,  0.9917476 ,  1.0584614 ,
        1.1740553 ,  1.3235219 ,  1.396176  ,  1.6066798 ,  1.8460381 ,
        2.0818584 ,         nan,         nan,  3.2851179 ,  3.9783525 ,
               nan,         nan,         nan,         nan,         nan,
               nan,  2.2683532 ,  2.2516294 ,         nan,         nan,
               nan,         nan,  1.4558617 ,  1.3435512 ,  1.4846169 ],
      dtype=float32)
Coordinates:
    lat           float64 8B 47.12
  * lon           (lon) float64 2kB 230.1 230.4 230.6 ... 299.4 299.6 299.9
    decade_month  <U7 28B '2050-07'

In [31]:
(0.47432232 + 0.41958573) /2

0.44695402500000003

In [70]:
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
from shapely import wkt

In [67]:
geometries = [
        Point(1, 1),
        Point(4, 4),
        Polygon([(2, 2), (2, 3), (3, 3), (3, 2)]),
        LineString([(0, 0), (1, 1), (2, 2), (2, 3)]),
    ]
df = pd.DataFrame({"osm_id": [1, 2, 3, 4], "geometry": geometries})
gdf = gpd.GeoDataFrame(df, geometry="geometry").set_index("osm_id")

In [79]:
data = np.array(
        [
            [
                [10.0,   20.0,   30.0,    40.0,     50.0],
                [100.0,  200.0,  300.0,   400.0,    500.0],
                [1000.0, 2000.0, 3000.0,  4000.0,   5000.0],
                [8.0,    9.0,    10.0,    11.0,     12.0],
                [13.0,   14.0,   15.0,    16.0,     17.0],
            ]
        ]
    )
times = ["2020-01"]
x = np.array([0, 1, 2, 3, 4])
y = np.array([0, 1, 2, 3, 4])
dims = ["decade_month", "y", "x"]
ds = xr.Dataset(
    data_vars={"value_mean": (dims, data),
                "value_median": (dims, data),
                "value_stddev": (dims, data),
                "value_min": (dims, data),
                "value_max": (dims, data),
                "value_q1": (dims, data),
                "value_q3": (dims, data)},
    coords={"decade_month": times, "y": y, "x": x}
)

In [76]:
data[0][2][3]

np.float64(4000.0)

In [80]:
ds.sel(x=2, y=3)

<xarray.Dataset> Size: 100B
Dimensions:       (decade_month: 1)
Coordinates:
  * decade_month  (decade_month) <U7 28B '2020-01'
    y             int64 8B 3
    x             int64 8B 2
Data variables:
    value_mean    (decade_month) float64 8B 10.0
    value_median  (decade_month) float64 8B 10.0
    value_stddev  (decade_month) float64 8B 10.0
    value_min     (decade_month) float64 8B 10.0
    value_max     (decade_month) float64 8B 10.0
    value_q1      (decade_month) float64 8B 10.0
    value_q3      (decade_month) float64 8B 10.0

In [39]:
values = []
for point in [(0, 0), (1, 1), (2, 2), (2, 3)]:
    dim = (0,) + point  # Create the tuple for indexing
    values.append(data[dim])  # Append the value

# Convert the list to a NumPy array
arr = np.array(values)


In [58]:
data[0][3][2]

np.float64(10.0)

In [42]:
arr

array([  10.,  200., 3000., 4000.])

In [46]:
7210/9

801.1111111111111